# IR Lab SoSe 2024: Combined Retrieval System

This jupyter notebook serves as an improved retrieval system combining components from both provided notebooks.
We will use a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This notebook serves as a retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 0: Install Required Packages and Setup Logging

Execute this cell if you're using Google Colab or if you haven't installed these packages yet.

In [ ]:
!pip install tira ir-datasets python-terrier transformers torch nltk

import logging
logging.basicConfig(level=logging.INFO)
logging.info("Logging initialized.")

### Step 1: Import Libraries

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
import os
from transformers import BertTokenizer, BertForTokenClassification, pipeline
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK data
nltk.download('punkt')

# Initialize PyTerrier and TIRA client
ensure_pyterrier_is_loaded()
tira = Client()

logging.info("Libraries imported successfully.")

### Step 2: Load the Dataset and the Index

In [ ]:
try:
    # The dataset: the union of the IR Anthology and the ACL Anthology
    pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
    logging.info("Dataset loaded successfully.")

    # A (pre-built) PyTerrier index loaded from TIRA
    index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
    logging.info("Index loaded successfully.")
except Exception as e:
    logging.error(f"An error occurred while loading the dataset or index: {str(e)}")
    raise

### Step 3: Define the Retrieval Pipeline

In [ ]:
# Base retrieval model with BM25
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

# Query expansion with Bo1
# fb_docs: number of feedback documents, fb_terms: number of expansion terms
bo1_expansion = pt.rewrite.Bo1QueryExpansion(index, fb_docs=10, fb_terms=20)
bm25_bo1 = bm25 >> bo1_expansion >> bm25

# Additional reranking models
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
dirichletLM = pt.BatchRetrieve(index, wmodel="DirichletLM")

# Combined retrieval pipeline
# We're giving more weight to TF-IDF and DirichletLM models
combined_pipeline = bm25_bo1 + 2 * tf_idf + 2 * dirichletLM

logging.info("Retrieval pipeline defined successfully.")

### Step 4: Create the Run

In [ ]:
print('First, we have a short look at the first three topics:')
topics = pt_dataset.get_topics('text')
print(topics.head(3))

# Query Segmentation
try:
    print('\nInitializing BERT model for Named Entity Recognition...')
    tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
    model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    logging.info('BERT model loaded successfully')
except Exception as e:
    logging.error(f"Error loading BERT model: {str(e)}")
    raise

# Domain-specific terms for query segmentation
domain_specific_terms = [
    "natural language processing", "NLP", "information retrieval", "IR",
    "machine learning", "deep learning", "neural network", "text mining",
    "language model", "BERT", "transformer", "word embeddings", "semantic search",
    "question answering", "text classification", "entity recognition",
    "tokenization", "part-of-speech tagging", "POS tagging", "named entity recognition", "NER",
    "sentiment analysis", "topic modeling", "latent Dirichlet allocation", "LDA",
    "vector space model", "TF-IDF", "BM25", "relevance feedback",
    "information retrieval evaluation", "precision", "recall", "F1 score",
    "mean average precision", "MAP", "normalized discounted cumulative gain", "nDCG",
    "word2vec", "GloVe", "fastText", "attention mechanism",
    "sequence-to-sequence", "seq2seq", "encoder-decoder", "automatic summarization",
    "machine translation", "language generation", "dialogue systems", "chatbots",
    "cross-lingual information retrieval", "multilingual models", "transfer learning",
    "fine-tuning", "pre-trained models", "zero-shot learning",
    "few-shot learning", "domain adaptation", "semi-supervised learning",
    "unsupervised learning", "self-supervised learning", "contrastive learning",
    "contextual embeddings", "contextualized word representations",
    "transformer-based models", "convolutional neural networks", "CNNs",
    "recurrent neural networks", "RNNs", "long short-term memory", "LSTM",
    "gated recurrent units", "GRU", "sequence labeling", "dependency parsing",
    "constituency parsing", "syntactic parsing", "semantic parsing",
    "coreference resolution", "relation extraction", "information extraction",
    "knowledge graphs", "ontologies", "semantic role labeling", "SRL",
    "document retrieval", "passage retrieval", "question answering systems",
    "retrieval-augmented generation", "RAG", "open-domain QA", "closed-domain QA",
    "query expansion", "query reformulation", "interactive information retrieval",
    "user modeling", "personalized search", "context-aware retrieval",
    "query understanding", "query intent", "search engine optimization", "SEO",
    "click-through rate", "CTR", "session-based search", "search result diversification",
    "exploratory search", "faceted search", "enterprise search",
    "legal information retrieval", "medical information retrieval",
    "scientific information retrieval", "scholarly search", "academic search",
    "digital libraries", "citation analysis", "bibliometrics", "altmetrics",
    "author disambiguation", "document clustering", "document classification",
    "information visualization", "search interfaces", "human-computer interaction",
    "HCI", "recommendation systems", "collaborative filtering", "content-based filtering",
    "hybrid recommendation", "ranking algorithms", "learning to rank", "LTR",
    "pairwise ranking", "listwise ranking", "pointwise ranking", "click models",
    "user feedback", "implicit feedback", "explicit feedback", "active learning",
    "crowdsourcing", "data annotation", "evaluation metrics", "benchmark datasets"
]

def advanced_segment_query(query):
    ner_results = nlp(query)
    segments = set(result['word'] for result in ner_results if result['entity'] in ['B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'])
    for term in domain_specific_terms:
        if term.lower() in query.lower():  # Case-insensitive matching
            segments.add(term)
    if not segments:
        segments = word_tokenize(query)
    return " ".join(segments)

print('\nSegmenting the queries...')
segmented_topics = topics.copy()
segmented_topics['query'] = segmented_topics['query'].apply(advanced_segment_query)
print(segmented_topics.head(3))

print('\nNow we do the retrieval...')
run = combined_pipeline.transform(segmented_topics)

print('\nDone. Here are the first 10 entries of the run')
print(run.head(10))

### Step 5: Persist the run file for subsequent evaluations

In [ ]:
try:
    os.makedirs('../runs', exist_ok=True)
    persist_and_normalize_run(run, system_name='combined-bm25-bo1-tfidf-dirichlet', default_output='../runs')
    output_dir = os.environ.get('outputDir', '/output')
    run.to_csv(os.path.join(output_dir, 'run.txt'), sep='\t', index=False, header=False)
    logging.info(f"Results saved to {os.path.join(output_dir, 'run.txt')}")
except Exception as e:
    logging.error(f"An error occurred while saving the run file: {str(e)}")
    raise